#### Copyright 2018 Google LLC.

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cat vs. Dog Image Classification


In [2]:
## Set These Parameters
last_layer_neurons = 1
last_layer_activation = "sigmoid"
loss_function = "binary_crossentropy"

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [5]:
!wget --no-check-certificate \
   https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O \
   /tmp/cats_and_dogs_filtered.zip

--2021-04-11 01:25:09--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.72.176, 142.250.188.240, 172.217.4.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.72.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  1.17MB/s    in 48s     

2021-04-11 01:25:59 (1.36 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [6]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
  
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

Next, let's apply the `datagen` transformations to a cat image from the training set to produce five random variants. Rerun the cell a few times to see fresh batches of random variants.

In [7]:
# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

#
class_mode = 'binary' if last_layer_neurons == 1 else 'categorical'

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode=class_mode)

# Flow validation images in batches of 32 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode=class_mode)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


If we train a new network using this data augmentation configuration, our network will never see the same input twice. However the inputs that it sees are still heavily intercorrelated, so this might not be quite enough to completely get rid of overfitting.

## Building a Small ConvNet


In [8]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from time import time
from tensorflow.keras.callbacks import TensorBoard

# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(img_input) 
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x) 
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 128 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Convolution2D(128, 3, activation='relu')(x) 
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x) 

# Add a dropout rate of 0.5
x = layers.Dropout(0.2)(x)

# add output layer
output = layers.Dense(last_layer_neurons, activation=last_layer_activation)(x)

# Configure and compile the model
model = Model(img_input, output)
model.compile(loss=loss_function,
              optimizer=optimizers.RMSprop(lr=1e-3),
              metrics=['acc'])

In [9]:
tensorboard = TensorBoard(log_dir='logs/number_of_last_layer_neurons{}last_layer_activation_{}_loss_function_{}_{}'.format(last_layer_neurons,last_layer_activation, loss_function, time()))
my_callbacks = [
      tensorboard,
]


history = model.fit(
      train_generator,
      epochs=27,
      validation_data=validation_generator,
      verbose=1,
      callbacks=my_callbacks)


Epoch 1/50
  1/100 [..............................] - ETA: 0s - loss: 0.7016 - acc: 0.4500WARNING:tensorflow:From /home/mojtaba/anaconda3/envs/deep_ai/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
100/100 [==============================] - 49s 489ms/step - loss: 0.8899 - acc: 0.5115 - val_loss: 0.6824 - val_acc: 0.5310
Epoch 2/50
100/100 [==============================] - 55s 546ms/step - loss: 0.6934 - acc: 0.6135 - val_loss: 0.6462 - val_acc: 0.6120
Epoch 3/50
100/100 [==============================] - 63s 626ms/step - loss: 0.6486 - acc: 0.6475 - val_loss: 0.6393 - val_acc: 0.6480
Epoch 4/50
100/100 [==============================] - 60s 604ms/step - loss: 0.6024 - acc: 0.6945 - val_loss: 0.6277 - val_acc: 0.6090
Epoch 5/50
100/100 [==============================] - 65s 650ms/step - loss: 

KeyboardInterrupt: 

Note that with data augmentation in place, the 2,000 training images are randomly transformed each time a new training epoch runs, which means that the model will never see the same image twice during training.

## Evaluate the Results

Let's evaluate the results of model training with data augmentation and dropout:

In [1]:
from matplotlib import pyplot as plt


# evaluate best model
%load_ext tensorboard
%tensorboard --logdir logs



# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

NameError: name 'history' is not defined

In [1]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from time import time
from tensorflow.keras.callbacks import TensorBoard
# evaluate best model
%load_ext tensorboard
%tensorboard --logdir logs